## 2.2  Setting Random Seeds
In this step, we fixed the related random seed to be used in order to obtain repeatable results. In this way, we have provided a deterministic path where we get the same result in every run. However, according to our observations, the results obtained on different computers may differ slightly (±1%)

In [1]:
## 2.2  Setting Random Seeds
seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)

## 2.3 Loading the data  
In this section, the files given for the train data are loaded.
- `wifi`  variable takes IDs and weights from the  `task2_train_estimated_wifi_distances.csv`.
- `steps`  variable takes IDs and weights from file `task2_train_steps.csv`.
- `elevs`  variable takes IDs from file  `task2_train_elevations.csv`.
- `fp_lookup`  variable gets  IDs and trajectories from the  `task2_train_lookup.json` file.


We did not prefer the method of recalculating the estimated distances given in wifi with the model we obtained in Task1, because the results we obtained from this process did not make a significant difference. That's why we didn't use the `task2_train_fingerprints.json` file in our ultimate work.




**Please make sure there are `task2_train_estimated_wifi_distances.csv`, `task2_train_steps.csv`, `task2_train_elevations.csv`, `task2_train_lookup.json`  files in the `task2_for_participants/train` folder!**

In [5]:
# unzip required files
import zipfile
with zipfile.ZipFile("task2_for_participants.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [6]:
## 2.3 Loading the data  
import os
import json
import csv
import networkx as nx
from tqdm import tqdm

path_to_data = "task2_for_participants/train"

with open(os.path.join(path_to_data,"task2_train_estimated_wifi_distances.csv")) as f:
    wifi = []
    reader = csv.DictReader(f)
    for line in tqdm(reader):
        wifi.append([line['id1'],line['id2'],float(line['estimated_distance'])])
        
with open(os.path.join(path_to_data,"task2_train_elevations.csv")) as f:
    elevs = []
    reader = csv.DictReader(f)
    for line in tqdm(reader):
        elevs.append([line['id1'],line['id2']])        

with open(os.path.join(path_to_data,"task2_train_steps.csv")) as f:
    steps = []
    reader = csv.DictReader(f)
    for line in tqdm(reader):
        steps.append([line['id1'],line['id2'],float(line['displacement'])]) 
        
fp_lookup_path = os.path.join(path_to_data,"task2_train_lookup.json")

with open(fp_lookup_path) as f:
    fp_lookup = json.load(f)

1944671it [00:04, 456463.28it/s]
2224it [00:00, 286701.87it/s]
98610it [00:00, 595575.40it/s]


## 2.3 Generating the Trajectory  graph. 
We take the average distance as the weight when creating the Trajectory graph. We used the example given for Task 2 (`Task2-IPS-Challenge-2021.ipynb`) for this process. We saved the resulting graph (`B`) as an adjacency list in the directory (as `my.adjlist`).

In [7]:
## 2.3 Generating the Trajectory  graph. 
B = nx.Graph()

# Get all the trajectory ids from the lookup
valid_nodes = set(fp_lookup.values())

for node in valid_nodes:
    B.add_node(node)

# Either add an edge or append the distance to the edge data
for id1,id2,dist in tqdm(wifi):
    if not B.has_edge(fp_lookup[str(id1)], fp_lookup[str(id2)]):
        
        B.add_edge(fp_lookup[str(id1)], 
                   fp_lookup[str(id2)], 
                   ty = "w", weight=[dist])
    else:
        B[fp_lookup[str(id1)]][fp_lookup[str(id2)]]['weight'].append(dist)
        
# Compute the mean edge weight
for edge in B.edges(data=True):
    B[edge[0]][edge[1]]['weight'] = sum(B[edge[0]][edge[1]]['weight'])/len(B[edge[0]][edge[1]]['weight'])
        
# If you have made a wifi connection between trajectories with an elev, delete the edge
for id1,id2 in tqdm(elevs):
    if B.has_edge(fp_lookup[str(id1)], fp_lookup[str(id2)]):
        B.remove_edge(fp_lookup[str(id1)], 
                      fp_lookup[str(id2)])

nx.write_adjlist(B, "my.adjlist")

100%|████████████████████████████████████████████████████████████████████████████████████| 2224/2224 [00:00<00:00, 1239618.88it/s]


## 2.4 Converting nodes to vectors
Before giving the adjacency list as input to the machine learning algorithms, we convert the nodes to vector. In our work we used Node2vec as a graph embedding algorithm methodology proposed by Grover&Leskovec in 2016 \[1\]. Node2vec is a semi-supervised algorithm for feature learning of the nodes in the network. Node2vec is created based on the skip-gram technique, which is an NLP approach motivated on Distributional Structure concept.According to idea, if different words used in similar contexts they probably have a similar meaning and there is an obvious relationship between them. Skip-gram technique uses center word(input) to predict neighbours(output) while computing the probabilities of surroundings based on a given window size (contiguous sequence of items before of after the center), in other words N-grams. Unlike the NLP approach, Node2vec system is not fed with words which have a linear structure, but nodes and edges, which have a distributed graphical structure. This multidimensional structure makes embeddings complex and computationally expensive, but Node2vec uses negative sampling with optimizing stochastic gradient descent (SGD) to deal with it. In addition to this, The random walk approach is used to detect sample neighboring nodes of the source node in a nonlinear structure. 

In our study, we first performed the vector representation of node relationships in low-dimensional space by modeling with node2vec from given distance of two nodes (weights). Then we used the output of Node2vec (graph embeddings), which has vectors of nodes, to feed traditional K-means clustering algorithm.

The parameters we use in Nod2vec can be listed as follows:

| Hyperparameter | Value |
| ------ | ------ |
|dimensions|32|
|walk_length|15|
|num_walks|100|
|workers|1|
|seed|0|
|window|10|
|min_count|1|
|batch_words|4|

The Node2vec model takes the adjacency list as input and outputs a 32-size vector. In this part, the node.py file is created and run in the `Jupyter Notebook`. There are two reasons why it is preferable to run externally rather than in a Jupyter notebook cell.
- It allows fixing the random seed of the node2vec module. Thus, it ensures repeatability by providing the same result in different runs.
- `Node2vec` is a very computationally expensive method, RAM overflow error is quite possible if run inside Jupyter notebook. Creating and running `Node2vec` model outside avoids this error.

The cell below creates the file named node.py. This file creates the node2vec model. This model takes the the adjacency list (`my.adjlist`) as input and creates a 32-dimensional vector file as output (`vectors.emb`).

**Important! The code below should be run in Linux distributions (Tested in Google colab and Ubuntu).**

In [8]:
# 2.4 Converting nodes to vectors
# A folder named tmp is created. This folder is essential for the node2vec model to use less RAM.
try:
    if not os.path.exists("tmp"):
        os.makedirs("tmp")
except OSError:
    print ("The folder could not be created!\n Please manually create the \"tmp\" folder in the directory")


node="""

# importing related modules

from node2vec import Node2Vec
import networkx as nx

#importing  adjacency list file as B
B = nx.read_adjlist("my.adjlist")

seed_value=0


# Specifying the input and hyperparameters of the node2vec model
node2vec = Node2Vec(B, dimensions=32, walk_length=15, num_walks=100, workers=1,seed=seed_value,temp_folder = './tmp')  

#Assigning/specifying random seeds

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)


# creation of the model

model = node2vec.fit(window=10, min_count=1, batch_words=4,seed=seed_value)   


# saving the output vector

model.wv.save_word2vec_format("vectors.emb")

# save the model
model.save("vectorMODEL")

"""
f = open("node.py", "w")
f.write(node)
f.close()


!PYTHONHASHSEED=0 python3 node.py 


'PYTHONHASHSEED' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!python node.py

## 2.4 Reshaping data
After our vector file is created, we read this file (`vectors.emb`). This file consists of 33 columns. The first column is the node number (IDs), and remain are vector values. By sorting the entire file by the first column, we return the nodes to their original order. Then we delete this ID column, which we will no longer use. So, we give the final shape of our data. Our data is ready to be used in machine learning applications.

In [10]:
# 2.4 Reshaping data
vec = np.loadtxt("vectors.emb", skiprows=1)
print("shape of vector file: ",vec.shape)
print(vec)
vec=vec[vec[:,0].argsort()]; 
vec=vec[0:vec.shape[0],1:vec.shape[1]]

shape of vector file:  (8371, 33)
[[ 4.8020000e+03  5.5105895e-01  1.8139237e-01 ...  2.0771873e-01
  -3.6370390e-01 -1.5758507e-01]
 [ 3.2080000e+03  6.6994180e-01  1.8356870e-02 ...  1.6239063e-01
  -3.3889225e-01 -3.2257378e-01]
 [ 3.7450000e+03  8.5945580e-01  1.7559928e-01 ...  3.2490490e-01
  -3.5132822e-01 -2.4129300e-01]
 ...
 [ 1.3850000e+03 -8.3527340e-03 -2.9195592e-02 ... -2.3357347e-02
   2.8256886e-03 -2.4400268e-02]
 [ 2.7150000e+03 -2.9293872e-02 -2.2110324e-02 ... -2.3724474e-03
   1.4017977e-02 -1.6061176e-02]
 [ 6.5080000e+03 -2.1313433e-02 -2.2092693e-02 ...  1.2072727e-02
   1.7553598e-02  1.3892725e-04]]


In [12]:
# 2.5  Best result
from sklearn import cluster
import time

ML_results = []
k_clusters = 17
algorithms = {}
algorithms['KMeans'] =cluster.KMeans(n_clusters=k_clusters,random_state=10)
second=time.time()
for model in algorithms.values():
    model.fit(vec)
    ML_results=list(model.labels_)
    print(model,time.time()-second)

KMeans(n_clusters=17, random_state=10) 2.140610456466675


## 2.6 RESULTS 
The output of the machine learning algorithm determines which cluster the fingerprints belong to. But what is required of us is to cluster the trajectories. Therefore, these fingerprints are converted to their trajectory counterparts using the `fp_lookup` variable. This output is processed into the `Node2Vec.csv` file.

In [13]:
## 2.6 RESULTS 
result={}
for ii,i in enumerate(set(fp_lookup.values())):
    result[i]=ML_results[ii]
        
    
ters={}
for i in result:
    if result[i] not in ters:
        ters[result[i]]=[]
        ters[result[i]].append(i)
    else:
        ters[result[i]].append(i)
        
        
        
final_results=[]
for i in ters:
    final_results.append(ters[i])
    
    
name="Node2Vec.csv"    
with open(name, "w", newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(final_results)
print(name, "file is ready!")

Node2Vec.csv file is ready!


-----------------------

## Runs the specified community detection algorithm and returns the results
- Fast Greedy
- Infomap
- Label Prop.
- Leiden
- Louvain

In [ ]:
import networkx as nx
import leidenalg
import igraph as ig
import time
import concurrent.futures
import sys
import csv
import os

def run_algorithm(G_ig, algorithm_name):
    """
    Runs the specified community detection algorithm and returns the results
    """
    start_time = time.time()
    
    try:
        if algorithm_name == "leiden":
            partition = leidenalg.find_partition(G_ig, leidenalg.ModularityVertexPartition)
        elif algorithm_name == "louvain":
            partition = G_ig.community_multilevel(weights='weight' if 'weight' in G_ig.edge_attributes() else None)
        elif algorithm_name == "label_propagation":
            partition = G_ig.community_label_propagation()
        elif algorithm_name == "fast_greedy":
            partition = G_ig.community_fastgreedy().as_clustering()
        elif algorithm_name == "infomap":
            partition = G_ig.community_infomap()
        elif algorithm_name == "edge_betweenness":
            # Edge betweenness is very slow for large graphs
            # Add a safety check here
            if G_ig.vcount() > 1000:
                print(f"WARNING: Edge Betweenness algorithm is very slow for graphs with {G_ig.vcount()} nodes.")
                print("Press 'e' to skip, or any other key to continue:")
                choice = input().lower()
                if choice == 'e':
                    return None, None
            partition = G_ig.community_edge_betweenness().as_clustering()
        else:
            print(f"Unknown algorithm: {algorithm_name}")
            return None, None
            
        elapsed_time = time.time() - start_time
        
        # Convert results to dictionary
        community_dict = {}
        for i, community in enumerate(partition):
            community_dict[i] = list(community)
            
        return community_dict, elapsed_time
        
    except Exception as e:
        print(f"An error occurred while running {algorithm_name}: {str(e)}")
        return None, None

def save_communities_to_csv(communities, algorithm_name):
    """
    Saves community results to a CSV file
    """
    if not communities:
        return False
    
    # Check/create output directory
    output_dir = "community_results"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create filename
    filename = os.path.join(output_dir, f"{algorithm_name.lower()}_communities.csv")
    
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # Write each community as a row
            for comm_id, members in communities.items():
                row = members
                writer.writerow(row)
        
        return True
    except Exception as e:
        print(f"Error while saving CSV file: {str(e)}")
        return False

def print_communities(communities, algorithm_name, elapsed_time):
    """
    Prints community results and saves them to a CSV file
    """
    if not communities:
        print(f"\n{algorithm_name} results could not be obtained.")
        return
        
    print(f"\n{algorithm_name} Results (Time: {elapsed_time:.2f} seconds):")
    print(f"Total of {len(communities)} communities detected.")
    
    # Show only the first 5 communities (in case there are too many)
    for i, (comm_id, members) in enumerate(communities.items()):
        if i < 5:
            print(f"Community {comm_id}: {len(members)} members")
        else:
            print("...")
            break
    
    # Find the largest community
    max_community = max(communities.items(), key=lambda x: len(x[1]))
    print(f"Largest community: Community {max_community[0]} ({len(max_community[1])} members)")
    
    # Save to CSV
    if save_communities_to_csv(communities, algorithm_name):
        print(f"Community results saved to {algorithm_name.lower()}_communities.csv")
    else:
        print(f"WARNING: {algorithm_name} results could not be saved to CSV.")

def main():
    # Check/create output directory
    output_dir = "community_results"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"'{output_dir}' directory created.")
    
    # Load the graph
    print("Loading graph...")
    start_time = time.time()
    
    try:
        G_nx = nx.read_adjlist("my.adjlist")
        
        # Print graph stats
        print(f"Graph properties:")
        print(f"Number of nodes: {G_nx.number_of_nodes()}")
        print(f"Number of edges: {G_nx.number_of_edges()}")
        
        # Convert to undirected if needed
        if nx.is_directed(G_nx):
            G_nx = G_nx.to_undirected()
        
        # Convert NetworkX graph to iGraph
        G_ig = ig.Graph.from_networkx(G_nx)
        
        elapsed_time = time.time() - start_time
        print(f"Graph loaded (Time: {elapsed_time:.2f} seconds)")
        
        # Choose algorithms to run
        algorithms = ["leiden", "louvain", "label_propagation", "fast_greedy", "infomap"]
        
        # Skip edge_betweenness for large graphs
        if G_ig.vcount() <= 1000:
            algorithms.append("edge_betweenness")
        else:
            print("\nNOTE: Edge Betweenness algorithm will not be run due to large graph size.")
        
        print("\nRunning community detection algorithms...")
        
        for algorithm in algorithms:
            print(f"\nRunning {algorithm.capitalize()} algorithm...")
            communities, elapsed_time = run_algorithm(G_ig, algorithm)
            if communities:
                print_communities(communities, algorithm.capitalize(), elapsed_time)
                
        # Alternatively, for parallel execution:
        """
        with concurrent.futures.ProcessPoolExecutor() as executor:
            futures = {executor.submit(run_algorithm, G_ig, alg): alg for alg in algorithms}
            
            for future in concurrent.futures.as_completed(futures):
                algorithm = futures[future]
                try:
                    communities, elapsed_time = future.result()
                    if communities:
                        print_communities(communities, algorithm.capitalize(), elapsed_time)
                except Exception as e:
                    print(f"Error while running {algorithm}: {str(e)}")
        """
        
        print("\nAll tasks completed.")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        
if __name__ == "__main__":
    main()
